In [19]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent

In [20]:
import os
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

In [21]:
agent = create_csv_agent(
    llm,
    "titanic.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True
)

In [22]:
agent.run("How many rows are there in dataset ?")



> Entering new AgentExecutor chain...
Thought: To find the number of rows in the dataframe, I can use the `len()` function or the `.shape` attribute. I'll use the `len()` function.
Action: python_repl_ast
Action Input: len(df)891I now know the final answer
Final Answer: 891

> Finished chain.


'891'

In [23]:
import json
## LLM as Judge
knowledge_base = []

message_countre = 0

while True:
    user_input = input("Enter your question : ")
    message_countre += 1
    user_message = f"user_msg_{message_countre} : {user_input}"
    knowledge_base.append(user_message)

    if user_input == "exit":
        break

    result = agent(user_input)
    agent_message = f"agent_msg_{message_countre} : {result}" 
    knowledge_base.append(agent_message)
    print(result)

## do prompt refinement for better answers as judge
llm_as_judge_prompt=f"""
You are a judge. You will output if the conversation is helpful or not based on the conversation and user experience.
return "helpful" or "not helpful" only no explanation needed.

Conversation:
{knowledge_base}
"""

print(knowledge_base)
verdict = llm.invoke(llm_as_judge_prompt)
print(verdict)




> Entering new AgentExecutor chain...
Thought: I need to calculate the average fare. I can use the `mean()` method on the 'Fare' column of the dataframe.
Action: python_repl_ast
Action Input: `df['Fare'].mean()`32.204207968574636I have calculated the average fare.
Final Answer: 32.204207968574636

> Finished chain.
{'input': 'avg price', 'output': '32.204207968574636'}
['user_msg_1 : avg price', "agent_msg_1 : {'input': 'avg price', 'output': '32.204207968574636'}", 'user_msg_2 : exit']
content='helpful' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-d77b0ca1-739e-4108-9fa7-b8365f8ac452-0' usage_metadata={'input_tokens': 101, 'output_tokens': 2, 'total_tokens': 103, 'input_token_details': {'cache_read': 0}}
